# Train a Single agent system using RLLib

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK

This notebook demonstrates how to use the ``PrimaiteRayEnv`` to train a basic PPO agent on the [UC2 scenario](./Data-Manipulation-E2E-Demonstration.ipynb).

In [ ]:
!primaite setup

In [ ]:
import yaml
import ray
from primaite.config.load import data_manipulation_config_path
from primaite.session.ray_envs import PrimaiteRayEnv
from ray.rllib.algorithms.ppo import PPOConfig

# If you get an error saying this config file doesn't exist, you may need to run `primaite setup` in your command line
# to copy the files to your user data path.
with open(data_manipulation_config_path(), 'r') as f:
    cfg = yaml.safe_load(f)

ray.init(local_mode=True)


#### Create a Ray algorithm and pass it our config.

In [ ]:
for agent in cfg['agents']:
    if agent["ref"] == "defender":
        agent['agent_settings']['flatten_obs'] = True
env_config = cfg

config = (
    PPOConfig()
    .environment(env=PrimaiteRayEnv, env_config=env_config)
    .env_runners(num_env_runners=0)
    .training(train_batch_size=128)
    .evaluation(evaluation_duration=1)
)


#### Start the training

In [ ]:
algo = config.build()
results = algo.train()


### Evaluate the results

In [ ]:
eval = algo.evaluate()